# Bayesian Mixture Model

In [17]:
# Add "beer" to the PYTHONPATH
import sys
sys.path.append('../')

import beer
import numpy as np

# For plotting.
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
output_notebook()

%load_ext autoreload
%autoreload 2

Loading BokehJS ...

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
def plot_normal(fig, mean, cov, n_sigma=2, alpha=1., color='blue'):
    'Plot a Normal density'
    # Eigenvalue decomposition of the covariance matrix.
    evals, evecs = np.linalg.eigh(cov)
    
    sign = 1 if cov[1, 0] == 0 else np.sign(cov[1, 0])
    # Angle of the rotation.
    angle =  - np.arccos(sign * abs(evecs[0, 0]))
   
    fig.ellipse(x=mean[0], y=mean[1], 
                width=n_sigma * 2* np.sqrt(evals[0]), 
                height=n_sigma * 2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
    fig.ellipse(x=mean[0], y=mean[1], 
                width=2 * np.sqrt(evals[0]), 
                height=2 * np.sqrt(evals[1]), 
                angle=angle, alpha=alpha, color=color)
    
def plot_mixture(fig, gmm, n_sigma=2, alpha=1., color='blue'):
    weights = gmm.weights
    for i, component in enumerate(gmm.components):
        plot_normal(fig, component.mean, component.cov, n_sigma, alpha * weights[i], color)

Generate some normally distributed data:

In [19]:
fig = figure(
    title='Data',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)

mean = np.array([-1.5, 4]) 
cov = np.array([
    [.75, 0],
    [0, 2.]
])
data1 = np.random.multivariate_normal(mean, cov, size=500)

plot_normal(fig, mean, cov, alpha=0.1)

mean = np.array([5, 5]) 
cov = np.array([
    [2, 1],
    [1, .75]
])
data2 = np.random.multivariate_normal(mean, cov, size=500)

plot_normal(fig, mean, cov, alpha=0.1)

data = np.vstack([data1, data2])
print(data.shape)
#np.random.shuffle(data)

fig.circle(data[:, 0], data[:, 1], alpha=.1)

show(fig)

(1000, 2)


In [20]:
p_mean = data.mean(axis=0)
p_cov = np.cov(data.T)
args = {'dim':2, 'mean':p_mean, 'cov': p_cov, 'prior_count':1e-6, 'random_init':True}
gmm_diag = beer.Mixture.create(10, beer.NormalDiagonalCovariance.create, args, prior_count=1e-6)
gmm_full = beer.Mixture.create(10, beer.NormalFullCovariance.create, args, prior_count=1e-6)
gmm_diag.fit(data, max_epochs=100)
gmm_full.fit(data, max_epochs=100, verbose=True)

ln p(X) = -2,900,000.804 E[ln p(X|...)] = -2,900,000.804 D(q || p) = 0.000
ln p(X) = -2.692       E[ln p(X|...)] = -2.187       D(q || p) = 0.505
ln p(X) = -2.518       E[ln p(X|...)] = -2.002       D(q || p) = 0.516
ln p(X) = -2.358       E[ln p(X|...)] = -1.831       D(q || p) = 0.527
ln p(X) = -2.246       E[ln p(X|...)] = -1.710       D(q || p) = 0.535
ln p(X) = -2.195       E[ln p(X|...)] = -1.657       D(q || p) = 0.538
ln p(X) = -2.172       E[ln p(X|...)] = -1.635       D(q || p) = 0.537
ln p(X) = -2.162       E[ln p(X|...)] = -1.628       D(q || p) = 0.534
ln p(X) = -2.155       E[ln p(X|...)] = -1.624       D(q || p) = 0.530
ln p(X) = -2.148       E[ln p(X|...)] = -1.622       D(q || p) = 0.526
ln p(X) = -2.139       E[ln p(X|...)] = -1.620       D(q || p) = 0.520
ln p(X) = -2.122       E[ln p(X|...)] = -1.618       D(q || p) = 0.504
ln p(X) = -2.093       E[ln p(X|...)] = -1.617       D(q || p) = 0.476
ln p(X) = -2.086       E[ln p(X|...)] = -1.615       D(q || p) = 0.471
ln

In [16]:
fig1 = figure(
    title='GMM (diag)',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)
fig1.circle(data[:, 0], data[:, 1], alpha=.1)
plot_mixture(fig1, gmm_diag, color='red')

fig2 = figure(
    title='GMM (full)',
    x_range=(-10, 10),
    y_range=(-10, 10),
    width=400,
    height=400,
)
fig2.circle(data[:, 0], data[:, 1], alpha=.1)
plot_mixture(fig2, gmm_full, color='green')

grid = gridplot([[fig1, fig2]])
show(grid)